In [7]:
import math
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import accuracy_score, matthews_corrcoef, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTETomek
import MachineLearning as ML
import PrepareData
import importlib

importlib.reload(ML)
importlib.reload(PrepareData)

<module 'PrepareData' from 'D:\\Studia\\Magisterka semestr 2\\Bazy i źródła danych\\Projekt\\WeatherPredictionProject\\PrepareData.py'>

In [3]:
# method 'teach_models' takes the data and tech all models with it

def teach_models(X_train, y_train):
    models=[]

    X_train, y_train = SMOTETomek().fit_resample(X_train, y_train)

    models.append(ML.NeuralNetworks(X_train, y_train))
    models.append(ML.KNN(X_train, y_train))
    models.append(ML.DecisionTree(X_train, y_train))
    models.append(ML.LinearRegressionModel(X_train, y_train))
    models.append(ML.LogisticRegressionModel(X_train, y_train))
    models.append(ML.RandomForest(X_train, y_train))
    models.append(ML.SupportVectorMachines(X_train, y_train))
    models.append(ML.SupportVectorRegression(X_train, y_train))
    models.append(ML.TreeGradientBoosting(X_train, y_train))
    models.append(ML.DummyModel(X_train, y_train))

    return models

In [4]:
# method 'score_models' calculate scores to many statistics for all models separately

def score_models(models, X_test, y_test):
    results=[]
    for model in models:
        scores={}
        y_pred=model.predict(X_test)>0.5
        scores['Model']=model.__str__()
        scores['Accuracy']=accuracy_score(y_test, y_pred)
        scores['F1_score']=f1_score(y_test, y_pred)
        scores['Matthwes_correlation_coefficient_(MCC)']=matthews_corrcoef(y_test, y_pred)
        scores['Mean_squared_error_(MSE)']=mean_squared_error(y_test, y_pred)
        scores['Square_root_of_mean_squared_error_(RMSE)']=math.sqrt(mean_squared_error(y_test, y_pred))
        scores['Mean_absolute_error_(MAE)']=mean_absolute_error(y_test, y_pred)
        scores['Confusion_matrix']=confusion_matrix(y_test, y_pred)

        results.append(scores)
    return results

In [5]:
# method 'score_merged_models' calculate scores to many statistics for selected models as merged object

def score_merged_models(models, X_test, y_test):
    merged_scores={}
    results=pd.DataFrame()
    num=1
    for model in models:
        column="Model_"+str(num)
        results[column]=list(model.predict(X_test)>0.5)
        num+=1

    results=results.astype(np.int32)

    results["Sum"]=results.sum(axis=1)/len(models)>0.5

    y_pred_merged=results["Sum"].values>0.5

    merged_scores['Model']='Merged model'
    merged_scores['Accuracy']=accuracy_score(y_test, y_pred_merged)
    merged_scores['F1_score']=f1_score(y_test, y_pred_merged)
    merged_scores['Matthwes_correlation_coefficient_(MCC)']=matthews_corrcoef(y_test, y_pred_merged)
    merged_scores['Mean_squared_error_(MSE)']=mean_squared_error(y_test, y_pred_merged)
    merged_scores['Square_root_of_mean_squared_error_(RMSE)']=math.sqrt(mean_squared_error(y_test, y_pred_merged))
    merged_scores['Mean_absolute_error_(MAE)']=mean_absolute_error(y_test, y_pred_merged)
    merged_scores['Confusion_matrix']=confusion_matrix(y_test, y_pred_merged)

    return merged_scores

In [8]:
# cell with data read, split it into train and test parts and teach the models

df=PrepareData.get_data()

df=df.dropna()

y=df['nextday_rainfall']
X=df.drop('nextday_rainfall', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.25, stratify=y)

models=teach_models(X_train, y_train)

Epoch 1/25
322/322 [==============================] - 2s 4ms/step - loss: 0.6938 - f1: 0.4130 - accuracy: 0.5006 - val_loss: 0.6900 - val_f1: 0.4950 - val_accuracy: 0.6260
Epoch 2/25
322/322 [==============================] - 1s 3ms/step - loss: 0.6909 - f1: 0.5573 - accuracy: 0.5261 - val_loss: 0.6806 - val_f1: 0.6159 - val_accuracy: 0.6721
Epoch 3/25
322/322 [==============================] - 1s 3ms/step - loss: 0.6720 - f1: 0.5858 - accuracy: 0.5757 - val_loss: 0.6358 - val_f1: 0.6855 - val_accuracy: 0.6908
Epoch 4/25
322/322 [==============================] - 1s 3ms/step - loss: 0.6556 - f1: 0.6749 - accuracy: 0.6233 - val_loss: 0.6239 - val_f1: 0.6993 - val_accuracy: 0.6954
Epoch 5/25
322/322 [==============================] - 1s 3ms/step - loss: 0.6514 - f1: 0.6820 - accuracy: 0.6402 - val_loss: 0.6177 - val_f1: 0.7105 - val_accuracy: 0.6902
Epoch 6/25
322/322 [==============================] - 1s 3ms/step - loss: 0.6493 - f1: 0.6792 - accuracy: 0.6449 - val_loss: 0.6227 - val_f1

In [9]:
# simple usage of 'score_models' method

scores=score_models(models,X_test,y_test)
scores[1]

63/63 [==============================] - 0s 1ms/step


{'Model': 'KNeighborsClassifier()',
 'Accuracy': 0.6611362493715435,
 'F1_score': 0.7022968197879859,
 'Matthwes_correlation_coefficient_(MCC)': 0.32147590172514406,
 'Mean_squared_error_(MSE)': 0.33886375062845653,
 'Square_root_of_mean_squared_error_(RMSE)': 0.5821200482962742,
 'Mean_absolute_error_(MAE)': 0.33886375062845653,
 'Confusion_matrix': array([[520, 242],
        [432, 795]], dtype=int64)}

In [10]:
# simple usage of 'score_merged_models' method

scores=score_merged_models(models,X_test,y_test)
scores

63/63 [==============================] - 0s 1ms/step


{'Model': 'Merged model',
 'Accuracy': 0.7139265962795375,
 'F1_score': 0.7512024486226497,
 'Matthwes_correlation_coefficient_(MCC)': 0.425134307760269,
 'Mean_squared_error_(MSE)': 0.2860734037204625,
 'Square_root_of_mean_squared_error_(RMSE)': 0.5348583024694135,
 'Mean_absolute_error_(MAE)': 0.2860734037204625,
 'Confusion_matrix': array([[561, 201],
        [368, 859]], dtype=int64)}

In [175]:
# The below algorithm allow determining which set of models is best (needs ~10 min)

best_score=0
best_models=[]
num=0

import itertools
for L in range(2,len(models) + 1):
    for subset in itertools.combinations(models[:-1], L):
        merged_scores=score_merged_models(list(subset),X_test,y_test)
        if merged_scores['F1_score']>best_score:
            best_score=merged_scores['F1_score']
            best_models=list(subset)
        if num%100==0:
            print(num)
        num+=1

print(best_score)
print(best_models)

63/63 [==============================] - 0s 2ms/step
0
63/63 [==============================] - 0s 2ms/step
100
63/63 [==============================] - 0s 2ms/step
200
63/63 [==============================] - 0s 2ms/step
300
63/63 [==============================] - 0s 2ms/step
400
63/63 [==============================] - 0s 2ms/step
500
63/63 [==============================] - 0s 2ms/step
0.7790839075800567
[DecisionTreeClassifier(), RandomForestClassifier(max_depth=3), LGBMClassifier()]
